In [ ]:
// Package references and using statements
#r "nuget:Microsoft.Azure.EventHubs, 4.1.0"
#r "nuget:Newtonsoft.Json, 12.0.3"
#r "nuget:Microsoft.Azure.EventHubs.Processor, 4.1.0"
using System;
using System.Threading;
using Microsoft.Azure.EventHubs;
using System.Dynamic;
using System.Text;
using Newtonsoft.Json;

In [ ]:
// Set up EventHubs client

EventHubClient eventHubClient;
var eventHubConnectionString = "Endpoint=sb://testsuffix-ns.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=2ZEATu8J4i5KZk1amnvaNemflRg8TNhJJYqk+qXpCdQ=";
var eventHubName = "test";
var connectionStringBuilder = new EventHubsConnectionStringBuilder(eventHubConnectionString) 
{
    EntityPath = eventHubName
};

eventHubClient = EventHubClient.CreateFromConnectionString(connectionStringBuilder.ToString());

In [ ]:
// Arbitrarily post event hubs messages with a certain amount

List<dynamic> outputList = new List<dynamic>();
Random rng = new Random();
for( int i = 0; i < 20; i++) 
{
    for(int j = 0; j < 10; j++) 
    {
        dynamic entity = new ExpandoObject();
        entity.CustomerId = i;
        entity.PurchaseAmount = rng.Next(1, 10000);
        entity.PurchaseTime = DateTime.Now;
        entity.TenderType = entity.PurchaseAmount % 2 == 1 ? "CASH" : "CREDIT";

        var jsonMessage = JsonConvert.SerializeObject(entity);
        var encodedMessage = Encoding.UTF8.GetBytes(jsonMessage);
        var eventData = new EventData(encodedMessage);
        eventData.Properties.Add("Test", "Test string to see if properties even exist here");
        outputList.Add(entity);
        await eventHubClient.SendAsync(eventData);
    }
}

// We select the properties so we get a pretty table
display(outputList.Select(output => new {output.CustomerId, output.PurchaseAmount, output.PurchaseTime, output.TenderType}));